<a href="https://colab.research.google.com/github/jveilleux2314/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{'spark-3.1.2'}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-25 00:08:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.81MB/s    in 0.2s    

2021-06-25 00:08:07 (5.81 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
import pyspark.sql.functions as sql_function
from pyspark.sql.functions import count
# Read in the Review dataset as a DataFrame


In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.count()

2062167

In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00A5MRG6O|          Dad Is Fat|
|B004GUSG4M|Before I Go To Sl...|
|B002WLCKG2|Trial by Fire: A ...|
|B00F1KWDZO|    Elvis Sunny Bear|
|B004GB1FIK|Cassidy Jones and...|
|B00ALPRZXE|Please Don't Tick...|
|B0064J2B9Y|How To Get Your E...|
|B00EA9SELE|Stitch (Slasherto...|
|B00CL4HZIS|Binary Cycle: Dis...|
|B00DND202W|He Loves Lucy (Th...|
|B007FEF9C8|Death of a Hero: ...|
|B00F0ZZVZY|Basic Ballet Move...|
|B0076BT83Q|CARDIO SUCKS: The...|
|B001MSMULG|The Kill Artist (...|
|B004I8VGYO|Becoming Odyssa: ...|
|B0013TTJSG|Wild Card: An Eli...|
|B00ETR7S8M|Divine Interventi...|
|B00CMB9OV6|Profile of Evil (...|
|B00B4HAI2I|With the Lightnin...|
|B00CYB8CD0|Time to converse ...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGYFDX8QXKEIR|   33605939|B007KO2MLO|     328837464| 2013-09-09|
|R13CBGTMNV9R8Z|   34058393|B005FLODDE|     764276359| 2013-09-09|
| R7DRFHC0F71O0|   39601147|B00EA3L35O|     535606445| 2013-09-09|
|R27LUKEXU3KBXQ|   17351407|B00BL3JV50|     240053004| 2013-09-09|
|R1VXTPUYMNU687|   10463387|B00CXU7U80|     931529805| 2013-09-09|
|R30DKW1GJWLPZC|   50484904|B004EWGS5G|     442453110| 2013-09-09|
|R18DPFG2FALJI9|    7145636|B00BNRJAT6|     856774152| 2013-09-09|
|R24D677N5WBW5Q|    6285538|B007FZOXJM|       5589837| 2013-09-09|
|R2FCJ9BQLSIOR3|   10278048|B00B6AK7LU|     362701357| 2013-09-09|
|R1R6K4MAKDWTXI|   16568972|B00EVMMLU0|     342745087| 2013-09-09|
|R3R5DILCWM8J7B|   26001763|B00E4W4984|     186117550| 2013-09-09|
| RR5K72IZOCOFE|   48203259|B005A1JBB8|     947574172| 2013-09

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RGYFDX8QXKEIR|          4|            0|          0|   N|                N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|                Y|
| R7DRFHC0F71O0|          5|            0|          0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|          2|   N|                N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|                Y|
|R18DPFG2FALJI9|          5|            0|          0|   N|                Y|
|R24D677N5WBW5Q|          5|            0|          0|   N|                Y|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|                Y|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [21]:
read_df = review_id_df.limit(100)

In [23]:
new_df.count()

100

In [24]:
prod_df = review_id_df.limit(100)

In [25]:
cust_df = review_id_df.limit(100)

In [26]:
vin_df = review_id_df.limit(100)

In [27]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cpn20xjexxaa.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "Avairene14$", 
          "driver":"org.postgresql.Driver"}

In [29]:
# Write review_id_df to table in RDS
new_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [30]:
# Write products_df to table in RDS
# about 3 min
prod_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [31]:
# Write customers_df to table in RDS
# 5 min 14 s
cust_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [32]:
# Write vine_df to table in RDS
# 11 minutes
vin_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)